<a href="https://colab.research.google.com/github/AndrewCrossman/Graduate_Research/blob/main/BNL_Scripts/Convert_tth_to_xy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Convert_tth_to_xy**

*This code converts .chi files located in a google drive directory into .xy files*

In [1]:
from google.colab import drive  #allows downloading and accessing files on drive
from google.colab import files  #allows downloading and accessing files on drive
import shutil                   #allows uploading files to drive
import glob                     #useful for collecting and organizing files
import csv
import numpy as np              #math library
import pandas as pd             #data organization library
import scipy as sp              #math library
from scipy import optimize      #math fitting library
import matplotlib.pylab as plt  #allows for plotting
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
!ls '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration'

Cu1Ni2_mixed_200_20240611-224349_4467f1_primary-1_mean_q.chi
Cu1Ni2_mixed_200_20240611-224349_4467f1_primary-1_mean_tth.chi
Cu1Ni2_mixed_200_20240611-224349_4467f1_primary-1_mean_tth.xy
Cu1Ni2_mixed_200_20240611-225445_ae49b0_primary-1_mean_q.chi
Cu1Ni2_mixed_200_20240611-225445_ae49b0_primary-1_mean_tth.chi
Cu1Ni2_mixed_200_20240611-225445_ae49b0_primary-1_mean_tth.xy
Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-100_mean_q.chi
Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-100_mean_tth.chi
Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-100_mean_tth.xy
Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-101_mean_q.chi
Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-101_mean_tth.chi
Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-101_mean_tth.xy
Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-102_mean_q.chi
Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-102_mean_tth.chi
Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-102_mean_tth.xy
Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-103_

In [7]:
directory_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration'
tth_files = glob.glob(directory_path + '/*tth.chi')
print(len(tth_files))

2195


In [5]:
data = pd.read_csv(tth_files[0])
print(data.shape[0])  # Gives number of rows
print(data.shape[1])  # Gives number of columns
data

3000
1


,# chi_2theta chi_I
0,2.891794622277297014e-03 1.561303257942199707e+00
1,8.675383866831891042e-03 8.798288702964782715e-01
2,1.445897311138648507e-02 4.799991250038146973e-01
3,2.024256235594108083e-02 2.137174367904663086e+00
4,2.602615160049567139e-02 2.070718765258789062e+00
...,...
2995,1.732474158206328951e+01 -1.010480523109436035...
2996,1.733052517130784054e+01 2.106397002935409546e-01
2997,1.733630876055239511e+01 8.241237401962280273e-01
2998,1.734209234979694969e+01 1.506945788860321045e-01


In [6]:
for file in tth_files:
  xy_name = file.replace(".chi",".xy")
  data = pd.read_csv(file, skiprows=1, header=None, sep="\s+").astype(float)
  data.to_csv(xy_name, header=False, sep='\t', index=False)  #downloads csv file directly to drive with no header, tab separations, and no index
  #print(xy_name)

In [8]:
directory_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration'
tth_files = glob.glob(directory_path + '/*tth.xy')
print(len(tth_files))

2195


In [ ]:
'''
'''
data = pd.read_csv(tth_files[0], skiprows=1, header=None, sep="\s+").astype(float)

#data.drop(data.columns[[0]], axis=1, inplace=True)
#data[['a', 'b']] = data[data.columns[0]].str.split(' ', expand=True)
#data = data[data.columns[0]].str.split(' ', expand=True)
#data = float(data)
#data.to_csv('/content/drive/Shareddrives/BNL_June_2024/DataFiles/CeO2_PDF_600_1sec/integration/CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-93_mean_tth_v1.xy', header=False, sep='\t', index=False)  #downloads csv file directly to drive with no header, tab separations, and no index
data

,0,1
0,0.002892,4.127588
1,0.008675,2.539962
2,0.014459,1.858726
3,0.020243,2.373410
4,0.026026,0.444784
...,...,...
2995,17.324742,-1.386520
2996,17.330525,-0.493231
2997,17.336309,-0.926230
2998,17.342092,0.282369
